In [2]:
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [3]:
model_name = "llama-3.1-8b-instant"
model = ChatGroq(model=model_name)

In [9]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [10]:
def create_outline(state: BlogState) -> BlogState:
    title = state['title']

    prompt = f"Create a detailed outline for blog of this title: {title}"
    outline = model.invoke(prompt).content

    state['outline'] = outline

    return state

In [11]:
def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']

    prompt = f"Write 10 line blog on the title: \n {title} using the following outline: \n {outline}"
    content = model.invoke(prompt).content

    state['content'] = content
    return state
    

In [14]:
graph = StateGraph(BlogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

initial_state = {"title": "Rise of AI in Pakistan"}
output_state = workflow.invoke(initial_state)
print(output_state)

{'title': 'Rise of AI in Pakistan', 'outline': "**Title:** The Rise of AI in Pakistan: Trends, Opportunities, and Challenges\n\n**I. Introduction**\n\n- Brief overview of AI and its global impact\n- Importance of AI in Pakistan's economic and social growth\n- Purpose of the blog: to discuss the rise of AI in Pakistan, its trends, opportunities, and challenges\n\n**II. History of AI in Pakistan**\n\n- Early adoption of AI in Pakistan (1990s-2000s)\n- Key milestones and initiatives that led to the growth of AI in Pakistan\n- Government and private sector investments in AI research and development\n\n**III. Current Trends in AI in Pakistan**\n\n- AI adoption in various sectors:\n  - Healthcare\n  - Education\n  - Finance\n  - Transportation\n  - Agriculture\n- AI-based startups and innovation hubs in Pakistan\n- Government initiatives to promote AI adoption and innovation\n\n**IV. Opportunities in AI for Pakistan**\n\n- Job creation and skills development in AI and related fields\n- Econo